In [ ]:
from pyspark.sql import SparkSession

# spark = SparkSession\
#     .builder\
#     .appName("myApp")\
#     .config("spark.mongodb.input.uri", "mongodb://mongo:27017/yelp.startup_log")\
#     .config("spark.mongodb.output.uri", "mongodb://mongo:27017/local.startup_log")\
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
#     .config("spark.sql.repl.eagerEval.enabled", True)\
#     .getOrCreate()
spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("myApp")\
    .config("spark.mongodb.input.uri", "mongodb://192.168.0.86:27017/yelp.startup_log")\
    .config("spark.mongodb.output.uri", "mongodb://192.168.0.86:27017/local.startup_log")\
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
    .config("spark.sql.repl.eagerEval.enabled", True)\
    .getOrCreate()

In [ ]:
review_df = spark\
    .read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://192.168.0.86:27017/local.yelp")\
    .load()

business_df = spark\
    .read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://192.168.0.86:27017/local.business")\
    .load()


In [ ]:
from pyspark.sql.functions import explode,split,avg,count,round,col
from pyspark.sql.types import IntegerType

In [ ]:


business_df_ph = business_df.withColumn("postal_code_int", business_df["postal_code"].cast(IntegerType()))

# business_df_ph


In [ ]:
business_df_ph = business_df_ph.filter( (business_df_ph['postal_code_int'] >= 19102 ) &   (business_df_ph['postal_code_int'] <= 19154 ) )

bss_ph_list = list(
    business_df_ph.select('business_id').toPandas()['business_id']
)

# bss_ph_list = business_df_ph.select("business_id").rdd.flatMap(lambda x: x).collect()

tmp = business_df_ph.select("business_id","categories","postal_code").withColumn('category_name',explode(split('categories',',')))

# tmp

In [ ]:

review_df_tmp = review_df.filter(col("business_id").isin(bss_ph_list))

tmp = tmp.join(review_df_tmp, on="business_id",how="inner")\
    .groupBy("postal_code", "category_name")\
    .agg(\
        round(avg("stars"), 2).alias("mean_stars"), \
        count("*").alias("cnt_reviews"))\

tmp

postal_code,category_name,mean_stars,cnt_reviews
19147,Thai,3.84,1015
19107,Bars,3.83,32548
19104,Car Rental,3.82,38
19103,Cardio Classes,4.05,172
19146,Printing Services,4.45,11
19150,Pizza,2.0,47
19103,Specialty Schools,4.1,224
19103,Health & Medical,3.48,2865
19153,Restaurants,2.71,824
19102,Tobacco Shops,4.46,143


In [ ]:
# filter out row where "cnt_reviews"<200
tmp_filter = tmp.where(tmp["cnt_reviews"]>200)
tmp_filter

postal_code,category_name,mean_stars,cnt_reviews
19147,Thai,3.84,1015
19107,Bars,3.83,32548
19148,Beer,3.38,1102
19128,Local Services,3.79,758
19103,Specialty Schools,4.1,224
19103,Health & Medical,3.48,2865
19153,Restaurants,2.71,824
19119,American (New),3.8,1474
19147,Irish,3.83,240
19123,Bakeries,4.02,461


In [ ]:
# from pyspark.sql.functions import col,bround

# tmp = tmp.select("postal_code","category_name",round(tmp["mean_stars"], 2).alias("avg_stars"), "cnt_reviews").orderBy(col("cnt_reviews").desc())
# tmp

In [ ]:

tmp_filter.coalesce(1).write.option("header","true").format("csv").save("out22")